In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-240254
Azure region: westeurope
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-240254


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

instance_name = "udacity-cpu-instance"

try:
    cpu_instance = ComputeTarget(workspace=ws, name=instance_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_instance = ComputeTarget.create(ws, instance_name, compute_config)

cpu_instance.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0, 10),
        "--max_iter": choice(20, 40, 60, 80, 100)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', script='train.py', compute_target=cpu_instance, environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=10,
                    max_concurrent_runs=4)

In [4]:
from azureml.widgets import RunDetails

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0d306986-23c7-423d-af7e-bf7e40f4302a
Web View: https://ml.azure.com/runs/HD_0d306986-23c7-423d-af7e-bf7e40f4302a?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-240254/workspaces/quick-starts-ws-240254&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-08-14T14:04:44.039213][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-08-14T14:04:44.6552554Z][SCHEDULER][INFO]Scheduling job, id='HD_0d306986-23c7-423d-af7e-bf7e40f4302a_0' 
[2023-08-14T14:04:44.7894537Z][SCHEDULER][INFO]Scheduling job, id='HD_0d306986-23c7-423d-af7e-bf7e40f4302a_1' 
[2023-08-14T14:04:44.9022602Z][SCHEDULER][INFO]Scheduling job, id='HD_0d306986-23c7-423d-af7e-bf7e40f4302a_2' 
[2023-08-14T14:04:44.924738][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-08-14T14:04:45.1215371Z][SCHEDULER][INFO]Scheduling job, id='HD_0d306986-23c7-423d-af7e-bf7e40f430

{'runId': 'HD_0d306986-23c7-423d-af7e-bf7e40f4302a',
 'target': 'udacity-cpu-instance',
 'status': 'Completed',
 'startTimeUtc': '2023-08-14T14:04:43.0188Z',
 'endTimeUtc': '2023-08-14T14:22:21.356107Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e2f32f33-56d9-41dd-a7f5-1f103e2f829a',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_0d306986-23c7-423d-af7e-bf7e40f4302a_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_0d306986-23c7-423d-af7e-bf7e40f4302a_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'teleme

In [5]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()

print(best_run)
print(best_run_metrics)

# Downloading model from the run into the workspace
print(best_run.get_file_names())
best_run.download_file('outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_0d306986-23c7-423d-af7e-bf7e40f4302a_0,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 1.929480590860051, 'Max iterations:': 80, 'Accuracy': 0.9099295945617868}
['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-08-14-14', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [7]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
combined_df = pd.concat([x,y], axis=1)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=combined_df,
    label_column_name='y',
    n_cross_validations=5)

In [10]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e45579b5-a399-461f-aec0-7432622ad22e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/08/14 14:56:26 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-08-14:15:18:59,863 INFO     [explanation_client.py:334] Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e45579b5-a399-461f-aec0-7432622ad22e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_e45579b5-a399-461f-aec0-7432622ad22e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-08-14T14:43:22.694904Z',
 'endTimeUtc': '2023-08-14T15:17:19.990897Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-240254","workspace_name":"quick-starts-ws-240254","region":"westeurope","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"ena

In [15]:
# Retrieve and save your best automl model
best_run, fitted_model = automl_run.get_output()
print(best_run)
print()
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_e45579b5-a399-461f-aec0-7432622ad22e_20,
Type: None,
Status: Completed)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), rand

In [16]:
cpu_instance.delete()